In [ ]:
import requests
import oauth2
import json
import pickle 
import os

with open("twitter_auth.json", 'r') as f:
    auth_codes = json.load(f)
    
def load_pickle(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    return data
    
def add_to_pickle_file(fname, new_data):
    # Given a pickle filename, append to it the new data (in our case, a list of dicts)
   
    # Check that file is not empty, otherwise initialize
    if os.path.getsize(fname) > 0:      
        with open(fname, 'rb') as f:
            unpickler = pickle.Unpickler(f);
            data = unpickler.load()
    else:
        data = []
    
    data = data + new_data
    
    with open(fname, 'wb') as f:
        print(data)
        pickle.dump(data, f)

def oauth_req(url, http_method="GET", post_body=b"", http_headers=None):
    consumer = oauth2.Consumer(key=auth_codes["key"], secret=auth_codes["secret"])
    token = oauth2.Token(key="", secret="")
    client = oauth2.Client(consumer, token)
    resp, content = client.request( url, method=http_method, body=post_body, headers=http_headers )
    return content

byteResponse = oauth_req('https://api.twitter.com/1.1/search/tweets.json?q=a&lang=en&count=100&tweet_mode=extended', "GET")
tweets = json.loads(byteResponse)

In [ ]:
# Usage: 
# 0) Make empty file called "patrick_labels.pckl" in working directory
# 1) Select correct pickle file
# 2) Label tweets as appropriate
# 3) When finished, type "done" into the emotion box to automatically have tweets uploaded to the pickle file. 

#pickle_file = "ben_labels.pckl"
pickle_file = "patrick_labels.pckl"

labeledTweets = []

# p - positive, n - negative, i - informational, h - humor, a - advertisement, o - none of the above
for status in tweets['statuses']:
    text = status['full_text']
    # If this is a retweet, the full_text field gets cut off at 140 characters, 
    # so we have to get the whole text from the retweeted_status field
    if text[0:4] == 'RT @':
        text = status['retweeted_status']['full_text']
    print(text)
    emotion = input("\nWhat emotion is this? (type 'done' to save current progress to file) ")
    if emotion.lower() == "done":
        break
    tweet = {"status": text, "userId": status['user']['id'], "emotion": emotion}
    labeledTweets.append(tweet)

add_to_pickle_file(pickle_file, labeledTweets)
labeledTweets

In [ ]:
# Print data in your pickle file to date
data = load_pickle(pickle_file)
print(data)

In [ ]:
labeledTweets

In [ ]:
import pip

installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
     for i in installed_packages])
print(installed_packages_list)